# Import Libraries

In [1]:
import pandas as pd
import numpy as np

# Import Dataset

In [2]:
data1 = pd.read_excel('Fuzzy_Topsis1.xlsx', index_col=0)
data2 = pd.read_excel('Fuzzy_Topsis2.xlsx', index_col=0)
data3 = pd.read_excel('Fuzzy_Topsis3.xlsx', index_col=0)

In [3]:
data1

,Communication Skills,Knowledge,Time to Solve
Criterias,,,
A1,Average,Very High,High
A2,High,Very High,Average
A3,Very High,Average,Low
A4,Low,Average,Low


In [4]:
data2

,Communication Skills,Knowledge,Time to Solve
Criterias,,,
A1,High,Very High,High
A2,High,High,Average
A3,Very High,Average,Very Low
A4,Low,Average,Very Low


In [5]:
data3

,Communication Skills,Knowledge,Time to Solve
Criterias,,,
A1,Average,High,High
A2,High,Average,Average
A3,High,Average,Low
A4,Very Low,Low,Very Low


# Fuzzification

In [6]:
fuzzifier = {'Very Low':(1,1,1), 'Low':(2,3,4), 'Average':(4,5,6), 'High':(6,7,8), 'Very High':(9,9,9)}
fuzzifier

{'Very Low': (1, 1, 1),
 'Low': (2, 3, 4),
 'Average': (4, 5, 6),
 'High': (6, 7, 8),
 'Very High': (9, 9, 9)}

In [7]:
for i in data1.columns:
    data1[i] = data1[i].map(fuzzifier)
data1

,Communication Skills,Knowledge,Time to Solve
Criterias,,,
A1,"(4, 5, 6)","(9, 9, 9)","(6, 7, 8)"
A2,"(6, 7, 8)","(9, 9, 9)","(4, 5, 6)"
A3,"(9, 9, 9)","(4, 5, 6)","(2, 3, 4)"
A4,"(2, 3, 4)","(4, 5, 6)","(2, 3, 4)"


In [8]:
for i in data2.columns:
    data2[i] = data2[i].map(fuzzifier)
data2

,Communication Skills,Knowledge,Time to Solve
Criterias,,,
A1,"(6, 7, 8)","(9, 9, 9)","(6, 7, 8)"
A2,"(6, 7, 8)","(6, 7, 8)","(4, 5, 6)"
A3,"(9, 9, 9)","(4, 5, 6)","(1, 1, 1)"
A4,"(2, 3, 4)","(4, 5, 6)","(1, 1, 1)"


In [9]:
for i in data3.columns:
    data3[i] = data3[i].map(fuzzifier)
data3

,Communication Skills,Knowledge,Time to Solve
Criterias,,,
A1,"(4, 5, 6)","(6, 7, 8)","(6, 7, 8)"
A2,"(6, 7, 8)","(4, 5, 6)","(4, 5, 6)"
A3,"(6, 7, 8)","(4, 5, 6)","(2, 3, 4)"
A4,"(1, 1, 1)","(2, 3, 4)","(1, 1, 1)"


# Combine Decision Matrix

In [10]:
cols = list(data1.columns)
cols

['Communication Skills', 'Knowledge', 'Time to Solve']

In [11]:
def combine(i, j):
    a = min(data1.iloc[i,j][0], data2.iloc[i,j][0], data3.iloc[i,j][0])
    b = np.mean([data1.iloc[i,j][1], data2.iloc[i,j][1], data3.iloc[i,j][1]])
    c = max(data1.iloc[i,j][2], data2.iloc[i,j][2], data3.iloc[i,j][2])
    return np.array([a,b,c])

In [12]:
decision_matrix = pd.DataFrame(columns=cols)

nrows, ncols = data1.shape
for i in range(nrows):
    vals = dict()
    for j in range(ncols):
        vals[cols[j]] = combine(i,j)
        
    decision_matrix = decision_matrix.append(vals, ignore_index=True)
    
decision_matrix

,Communication Skills,Knowledge,Time to Solve
0,"[4.0, 5.666666666666667, 8.0]","[6.0, 8.333333333333334, 9.0]","[6.0, 7.0, 8.0]"
1,"[6.0, 7.0, 8.0]","[4.0, 7.0, 9.0]","[4.0, 5.0, 6.0]"
2,"[6.0, 8.333333333333334, 9.0]","[4.0, 5.0, 6.0]","[1.0, 2.3333333333333335, 4.0]"
3,"[1.0, 2.3333333333333335, 4.0]","[2.0, 4.333333333333333, 6.0]","[1.0, 1.6666666666666667, 4.0]"


# Normalize

In [14]:
beneficiary_or_not = {'Communication Skills' : 'B', 'Knowledge':'B', 'Time to Solve' :'NB'}
beneficiary_or_not

{'Communication Skills': 'B', 'Knowledge': 'B', 'Time to Solve': 'NB'}

In [15]:
max_val = 0
min_val = 999
for i in decision_matrix.columns:
    for j in decision_matrix[i]:
        if(beneficiary_or_not[i]=='B'):
            max_val = max(max(j), max_val)
        else:
            min_val = min(min(j), min_val)
                
    if(beneficiary_or_not[i]=='B'):
        decision_matrix[i] = decision_matrix[i] / max_val
        max_val = -1
    else:
        decision_matrix[i] = (min_val / decision_matrix[i])
        
decision_matrix['Time to Solve'] = decision_matrix['Time to Solve'].apply(lambda row:np.flip(row))

In [16]:
decision_matrix

,Communication Skills,Knowledge,Time to Solve
0,"[0.4444444444444444, 0.6296296296296297, 0.888...","[0.6666666666666666, 0.925925925925926, 1.0]","[0.125, 0.14285714285714285, 0.16666666666666666]"
1,"[0.6666666666666666, 0.7777777777777778, 0.888...","[0.4444444444444444, 0.7777777777777778, 1.0]","[0.16666666666666666, 0.2, 0.25]"
2,"[0.6666666666666666, 0.925925925925926, 1.0]","[0.4444444444444444, 0.5555555555555556, 0.666...","[0.25, 0.42857142857142855, 1.0]"
3,"[0.1111111111111111, 0.2592592592592593, 0.444...","[0.2222222222222222, 0.48148148148148145, 0.66...","[0.25, 0.6, 1.0]"


# Weights

In [23]:
w = np.array([(6,7,8), (9,9,9), (4,5,6)])
w

array([[6, 7, 8],
       [9, 9, 9],
       [4, 5, 6]])

In [19]:
decision_matrix / w

ValueError: Unable to coerce to DataFrame, shape must be (4, 3): given (3, 3)